# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,tikapur,NP,28.50,81.13,78.01,95,0,3.00,1602856784
1,1,kavieng,PG,-2.57,150.80,82.04,77,97,12.53,1602856855
2,2,busselton,AU,-33.65,115.33,55.99,51,0,14.12,1602856857
3,3,launceston,AU,-41.45,147.17,44.60,70,90,1.12,1602856592
4,4,lebu,CL,-37.62,-73.65,58.48,70,0,3.96,1602856859


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
humidity = cities_df["Humidity"].astype(float)


fig = gmaps.figure(zoom_level=1, center=(0, 0))

# Create a humidity Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# ideal weather conditions
ideal_max_temp = 75
ideal_min_temp = 65
ideal_wind_speed = 10
ideal_cloud = 0

# create dataframe to satisfy ideal weather conditions
vacation_df = cities_df.loc[(cities_df['Max Temp'] >= ideal_min_temp) &
                           (cities_df['Max Temp'] < ideal_max_temp) &
                           (cities_df['Wind Speed'] <= ideal_wind_speed) &
                           (cities_df['Cloudiness'] <= ideal_cloud)]

#drop any rows will null values
vacation_df.dropna()
vacation_df

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
44,44,herat,AF,34.50,62.00,68.00,19,0,9.17,1602856908
64,64,guerrero negro,MX,27.98,-114.06,71.13,70,0,9.06,1602856935
144,144,castro,BR,-24.79,-50.01,68.49,60,0,8.43,1602856851
326,326,rudbar,AF,30.15,62.60,66.79,22,0,8.08,1602857247
328,328,luderitz,NaN,-26.65,15.16,69.71,53,0,9.95,1602857249
350,350,romitan,UZ,39.93,64.38,68.00,30,0,6.93,1602857275
417,417,sabzevar,IR,36.21,57.68,71.60,15,0,6.93,1602857353
457,457,veinticinco de mayo,AR,-35.43,-60.17,73.00,68,0,4.00,1602857401
463,463,yeppoon,AU,-23.13,150.73,68.00,77,0,5.82,1602857410


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# add column for Hotel Name
hotel_df = pd.DataFrame({"City": vacation_df["City"],
                         "Country": vacation_df["Country"],
                         "Lat": vacation_df["Lat"],
                         "Lng": vacation_df["Lng"]
                        })
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# set the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



In [8]:
# for each city in the ideal vacation spots
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))